# Cross data matrix methodology

Methodology for high-dimensional and low-sample-size data (HDLSS).

### References
- https://www.sciencedirect.com/science/article/pii/S0047259X19301915 (Main reference, in particular 2.2. Cross data matrix method)
- https://www.sciencedirect.com/science/article/pii/S0047259X10000904


In [1]:
import numpy as np
import pandas as pd

from math import sqrt
from copy import deepcopy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils.extmath import svd_flip
from statsmodels.regression.quantile_regression import QuantReg

from src.meta_results_r_data import prepare_fforma_data

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
X = np.array([[7827.764, 8421.74 , 8422.   , 6535.873, 8750.611, 8422.   ,
        7827.764, 8577.088, 8487.396],
       [7333.335, 8421.74 , 8422.   , 6471.836, 9079.222, 8422.   ,
        7333.335, 8577.088, 8553.045],
       [6921.949, 8421.74 , 8422.   , 6462.348, 9407.833, 8422.   ,
        6921.949, 8577.088, 8618.694],
       [6579.659, 8421.74 , 8422.   , 6460.832, 9736.444, 8422.   ,
        6579.659, 8577.088, 8684.343]])

In [4]:
X_test = np.array([[16037.273, 16661.017, 15449.   , 21518.987, 16037.273, 15449.   ,
        16778.538, 15916.413, 15620.592],
       [16625.545, 17873.283, 15449.   , 29255.582, 16625.545, 15449.   ,
        16961.09 , 15916.413, 15792.501],
       [17213.818, 19085.549, 15449.   , 32706.167, 17213.818, 15449.   ,
        16335.008, 15916.413, 15964.41 ],
       [17802.091, 20297.815, 15449.   , 33172.966, 17802.091, 15449.   ,
        15752.033, 15916.413, 16136.319]])

In [5]:
X.shape

(4, 9)

### Issues with PCA

In [6]:
pca_or = PCA().fit(X)

In [7]:
pca_or.components_

array([[ 0.615,  0.   , -0.   ,  0.036, -0.484, -0.   ,  0.615, -0.   ,
        -0.097],
       [-0.296,  0.   , -0.   , -0.478, -0.757, -0.   , -0.296, -0.   ,
        -0.151],
       [-0.186,  0.   , -0.   ,  0.878, -0.393, -0.   , -0.186, -0.   ,
        -0.078],
       [-0.05 , -0.004,  0.001, -0.   ,  0.195,  0.   ,  0.05 ,  0.   ,
        -0.978]])

In [8]:
pca_or.transform(X)

array([[1063.657,  -28.976,    1.354,    0.   ],
       [ 288.33 ,   35.398,   -4.891,    0.   ],
       [-382.96 ,   24.557,    5.797,    0.   ],
       [-969.026,  -30.978,   -2.26 ,    0.   ]])

In [9]:
pca_or.transform(X_test)

array([[  7935.364, -18866.516,   6531.23 ,  -5548.782],
       [  8385.266, -23265.036,  12932.431,  -5627.267],
       [  8184.59 , -25375.126,  15723.   ,  -5745.786],
       [  7903.506, -26071.375,  15887.07 ,  -5862.171]])

### New methodology

In [10]:
X_wm = X - X.mean(0) 

In [11]:
X1, X2 = np.split(X_wm, 2)

In [12]:
print('X1 shape', X1.shape)
print('X2 shape', X2.shape)

X1 shape (2, 9)
X2 shape (2, 9)


In [13]:
n1, n2 = X1.shape[0], X2.shape[0]

In [14]:
S1 = X1 @ X2.T / sqrt(n1)

In [15]:
print('S1 shape', S1.shape)

S1 shape (2, 2)


#### Getting eigenvalues and eigenvectors

In [16]:
u1, lambdas, u2 = np.linalg.svd(S1)

### Getting PCs

In [17]:
h1 = X1.T @ u1 / np.sqrt(n1 * lambdas)
h2 = X2.T @ u2 / np.sqrt(n2 * lambdas)

In [18]:
h = 0.5 * (h1 + h2)

In [20]:
h = h / np.linalg.norm(h, axis=0) 

In [21]:
pc_cross_data = X_wm @ h

# Comparison

In [24]:
print('PCA train:\n', pca_or.transform(X))
print('Cross-data-matrix train:\n', pc_cross_data)
print('PCA test:\n', pca_or.transform(X_test))
print('Cross-data-matrix test:\n', (X_test - X.mean(0)) @ h)

PCA train:
 [[1063.657  -28.976    1.354    0.   ]
 [ 288.33    35.398   -4.891    0.   ]
 [-382.96    24.557    5.797    0.   ]
 [-969.026  -30.978   -2.26     0.   ]]
Cross-data-matrix train:
 [[-1063.642   -31.923]
 [ -288.347    34.514]
 [  382.948    25.711]
 [  969.042   -28.302]]
PCA test:
 [[  7935.364 -18866.516   6531.23   -5548.782]
 [  8385.266 -23265.036  12932.431  -5627.267]
 [  8184.59  -25375.126  15723.     -5745.786]
 [  7903.506 -26071.375  15887.07   -5862.171]]
Cross-data-matrix test:
 [[ -7925.748 -18787.896]
 [ -8373.388 -23090.555]
 [ -8171.628 -25157.723]
 [ -7890.19  -25850.629]]


In [25]:
print('Rotation PCA:\n', pca_or.components_)
print('Rotation cross-data-matrix:\n', h.T)

Rotation PCA:
 [[ 0.615  0.    -0.     0.036 -0.484 -0.     0.615 -0.    -0.097]
 [-0.296  0.    -0.    -0.478 -0.757 -0.    -0.296 -0.    -0.151]
 [-0.186  0.    -0.     0.878 -0.393 -0.    -0.186 -0.    -0.078]
 [-0.05  -0.004  0.001 -0.     0.195  0.     0.05   0.    -0.978]]
Rotation cross-data-matrix:
 [[-0.614  0.     0.    -0.036  0.484  0.    -0.614  0.     0.097]
 [-0.3    0.     0.    -0.465 -0.762  0.    -0.3    0.    -0.152]]


# FQRA

### CrossDataMatrix class

In [26]:
class CrossDataMatrix:
    
    def __init__(self, n_components):
        self.n_components = n_components
    
    def _fit(self, X, n_components):
        # Center data
        X = deepcopy(X) # Avoid overwrite original X
        self.mean_ = np.mean(X, axis=0)
        X -= self.mean_
        
        X1, X2 = np.split(X, 2)
        n1, n2 = X1.shape[0], X2.shape[0]
        
        S1 = X1 @ X2.T / sqrt(n1)
        
        u1, lambdas, u2 = np.linalg.svd(S1)

        h1 = X1.T @ u1 / np.sqrt(n1 * lambdas)
        h2 = X2.T @ u2 / np.sqrt(n2 * lambdas)
        
        h = 0.5 * (h1 + h2)
        h = h / np.linalg.norm(h, axis=0)
        
        self.components_ = h[:, :n_components]
        
        return h
    
    def fit(self, X):
        self._fit(X, self.n_components)
        
        return self
    
    def transform(self, X):
        X = deepcopy(X)
        if self.mean_ is not None:
            X -= self.mean_
        X_transformed = X @ self.components_
        
        return X_transformed

In [27]:
pca = CrossDataMatrix(n_components=1).fit(X)

In [28]:
pca.transform(X)

array([[-1063.642],
       [ -288.347],
       [  382.948],
       [  969.042]])

# Experiments

In [29]:
data = prepare_fforma_data(directory='data', dataset_name=None, kind='TOURISM')


% of missing values, train set
x_acf10               7.551487
diff1_acf1            7.551487
diff1_acf10           7.551487
diff2_acf1            7.551487
diff2_acf10           8.543097
seas_acf1            39.511823
arch_lm               9.687262
garch_acf             0.076278
arch_r2              13.196034
garch_r2             13.348589
nonlinearity          1.144165
x_pacf5               1.144165
diff1x_pacf5          1.830664
diff2x_pacf5          2.212052
seas_pacf            39.511823
e_acf10               7.551487
seasonal_strength    39.511823
peak                 39.511823
trough               39.511823
hw_alpha             39.511823
hw_beta              39.511823
hw_gamma             39.511823
dtype: float64


% of missing values, test set
x_acf10               1.830664
diff1_acf1            1.830664
diff1_acf10           2.212052
diff2_acf1            2.212052
diff2_acf10           7.551487
seas_acf1            39.511823
arch_lm               7.551487
arch_r2               7

In [30]:
X_train = data['preds_train_df']
y_train = data['y_train_df']
X_test = data['preds_test_df']
y_test = data['y_test_df']

### FQRA using CrossDataMatrix

In [31]:
unique_ids = X_train['unique_id'].unique()
smape_list = []
y_hat_list = []
u_id_list = []
ds_list = []
params = []
for u_id in unique_ids:
    x_id = X_train[X_train['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    x_test = X_test[X_test['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    y_id = y_train[y_train['unique_id']==u_id]['y'].values
    y_test_id = y_test[y_test['unique_id']==u_id]['y'].values
    
    pca_model = CrossDataMatrix(n_components=1).fit(x_id)
    X = pca_model.transform(x_id)
    X = np.hstack([X, np.ones((len(X),1))])

    reg = QuantReg(y_id, X).fit(0.5)
    params.append(reg.params)

    x_test_pca = pca_model.transform(x_test)
    x_test_pca = np.hstack([x_test_pca, np.ones((len(x_test_pca),1))])
    y_hat = reg.predict(x_test_pca)

    my_smape=200*np.mean(np.abs(y_hat-y_test_id)/(np.abs(y_hat)+np.abs(y_test_id)))
    smape_list.append(my_smape)
    y_hat_list += y_hat.tolist()
    u_id_list += [u_id]*len(y_hat)
    ds_list += list(range(len(y_hat)))
    
df_hat = pd.DataFrame(list(zip(u_id_list, ds_list, y_hat_list)), 
               columns =['unique_id', 'ds', 'y_hat']) 
    
print('PCA for HDLSS:', np.mean(smape_list))
print('Note: omitting normalization of h inside CrossDataMatrix gets ~29.')

PCA for HDLSS: 30.36310792719511
Note: omitting normalization of h inside CrossDataMatrix gets ~29.


### FQRA using PCA

In [32]:
unique_ids = X_train['unique_id'].unique()
smape_list = []
y_hat_list = []
u_id_list = []
ds_list = []
for u_id in unique_ids:
    x_id = X_train[X_train['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    x_test = X_test[X_test['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    y_id = y_train[y_train['unique_id']==u_id]['y'].values
    y_test_id = y_test[y_test['unique_id']==u_id]['y'].values
       
    pca_model = PCA(n_components=1).fit(x_id)
    X = pca_model.transform(x_id)
    X = np.hstack([X, np.ones((len(X),1))])

    reg = QuantReg(y_id, X).fit(0.5)

    x_test_pca = pca_model.transform(x_test)
    x_test_pca = np.hstack([x_test_pca, np.ones((len(x_test_pca),1))])
    y_hat = reg.predict(x_test_pca)

    my_smape=200*np.mean(np.abs(y_hat-y_test_id)/(np.abs(y_hat)+np.abs(y_test_id)))
    smape_list.append(my_smape)
    y_hat_list += y_hat.tolist()
    u_id_list += [u_id]*len(y_hat)
    ds_list += list(range(len(y_hat)))
    
df_hat = pd.DataFrame(list(zip(u_id_list, ds_list, y_hat_list)), 
               columns =['unique_id', 'ds', 'y_hat']) 
    
print('PCA', np.mean(smape_list))

PCA 30.24614997941319
